# TRAINING / FINE-TUNING UNWEIGHTED BINARY BERT CLASSIFIER

In [1]:
# Initialise relevant packages

# Basics
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
# Preprocessing
import torch
from sklearn.model_selection import train_test_split

# Modelling
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

# Evaluation
from sklearn.metrics import classification_report, f1_score

## Load Datasets

The new method is described in the following section as it takes into account the pre-made data splits

In [2]:
## Load datasets
#training_data = pd.read_pickle('./Data/Clean Training Data/training_data_binary.pkl')
#
#df_raw = {}
#
## write to dict
#for dataset in training_data:
#    df_raw[dataset] = training_data[dataset].copy()#.sample(n=100, random_state=123)


# FOR NOW: Train just on "combined" dataset 

## delete all but "combined" dict entries
#for model in ['davidson2017', 'founta2018', 'dynabench2021']:
#    del df_raw[model]

## Split each dataset into training and validation set
#df_train, df_valtest, df_val, df_test = {}, {}, {}, {}
#
#for dataset in df_raw:
#    df_train[dataset], df_valtest[dataset] = train_test_split(df_raw[dataset], test_size=0.2, stratify=df_raw[dataset].label, random_state=123)
#    df_val[dataset], df_test[dataset] = train_test_split(df_valtest[dataset], test_size=0.5, stratify=df_valtest[dataset].label, random_state=123)

## Split up text and label columns in dataframes into series for each dataset
#train_texts, val_texts, test_texts, train_labels, val_labels, test_labels = {}, {}, {}, {}, {}, {}
#
#for dataset in df_raw:
#    train_texts[dataset] = df_train[dataset].text.astype("string").tolist()
#    val_texts[dataset] = df_val[dataset].text.astype("string").tolist()
#    test_texts[dataset] = df_test[dataset].text.astype("string").tolist()
#    
#    train_labels[dataset] = df_train[dataset].label.tolist()
#    val_labels[dataset] = df_val[dataset].label.tolist()
#    test_labels[dataset] = df_test[dataset].label.tolist()

## Load Datasets ( new method )

In [6]:
train_texts, val_texts, test_texts, train_labels, val_labels, test_labels = {}, {}, {}, {}, {}, {}
dataset_list = ['davidson2017', 'founta2018', 'dynabench2021','combined']
# write to dict
for dataset in dataset_list:
    data_train = pd.read_csv(f"/home/ec2-user/Projects/v0.1/Data/{dataset}/{dataset}_train.csv",lineterminator="\n")#training_data[dataset].copy()#.sample(n=100, random_state=123)#df_train[dataset].text.astype("string").tolist()
    data_test = pd.read_csv(f"/home/ec2-user/Projects/v0.1/Data/{dataset}/{dataset}_test.csv",lineterminator="\n")#training_data[dataset].copy()#.sample(n=100, random_state=123)#df_train[dataset].text.astype("string").tolist()
    data_val = pd.read_csv(f"/home/ec2-user/Projects/v0.1/Data/{dataset}/{dataset}_valid.csv",lineterminator="\n")#training_data[dataset].copy()#.sample(n=100, random_state=123)#df_train[dataset].text.astype("string").tolist()
    
    train_texts[dataset] = data_train.text.astype("string").tolist()
    val_texts[dataset] = data_val.text.astype("string").tolist()
    test_texts[dataset] = data_test.text.astype("string").tolist()
    
    train_labels[dataset] = data_train.label.tolist()
    val_labels[dataset] = data_val.label.tolist()
    test_labels[dataset] = data_test.label.tolist()

    
    
    

## Split up text and label columns in dataframes into series for each dataset
#train_texts, val_texts, test_texts, train_labels, val_labels, test_labels = {}, {}, {}, {}, {}, {}
#
#for dataset in df_raw:
#    train_texts[dataset] = df_train[dataset].text.astype("string").tolist()
#    val_texts[dataset] = df_val[dataset].text.astype("string").tolist()
#    test_texts[dataset] = df_test[dataset].text.astype("string").tolist()
#    
#    train_labels[dataset] = df_train[dataset].label.tolist()
#    val_labels[dataset] = df_val[dataset].label.tolist()
#    test_labels[dataset] = df_test[dataset].label.tolist()

In [7]:
train_texts.keys()

dict_keys(['davidson2017', 'founta2018', 'dynabench2021', 'combined'])

In [8]:
train_texts['davidson2017']

['rt [USER]: s/o to them side hos that know they the side hos we need more of ya!',
 "[USER] [USER] but s he got a pussy now which makes him a her. and she's bad. so..",
 'you been the side hoe 😂😂😂💀💀👊',
 'rt [USER]: all you hoes wake up',
 'rt [USER] this whole week was trash',
 'soap. i wish some pussy tho rt [USER]: what does your beard smell like?',
 "in case you forgot hoe i been up since 4:18 on the dot made 100,000 dollars at work and still haven't slept.",
 'rt [USER]: austin is so cute like look at him all excited because he got oreos [URL]',
 'rt [USER]: 😂😂 hoes be in everybody clothes and got the nerve to take a picture man that shit crazy !',
 'i swear ta god nigga all deses hoes want is a yrn ....',
 '[USER] lol true i just want fried oreos',
 '"you got hella hoes" [URL]',
 "[USER] say how you do that i got this lil bitch who ain't getting the memo",
 '“[USER]: “[USER]: bout to get a breasts reduction these bitches be in the way😒”😂😂😂”so serious😩😩😩',
 '[USER] pussy',
 'abed 

In [9]:
train_labels['davidson2017']

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


## Tokenize Texts

In [10]:
# import tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# add special tokens for URLs, emojis and mentions (--> see pre-processing)
special_tokens_dict = {'additional_special_tokens': ['[USER]','[EMOJI]','[URL]']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

In [11]:
# Tokenize text series for each dataset
train_encodings, val_encodings, test_encodings = {}, {}, {}

for dataset in dataset_list:
    train_encodings[dataset] = tokenizer(train_texts[dataset], truncation=True, padding=True,max_length = 64)
    val_encodings[dataset] = tokenizer(val_texts[dataset], truncation=True, padding=True,max_length = 64)
    test_encodings[dataset] = tokenizer(test_texts[dataset], truncation=True, padding=True,max_length = 64)
    
# sanity check for length of encoding vectors
len(train_encodings[dataset]["input_ids"][0])

64

## Create PyTorch Datasets 

In [ ]:
class HateDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset, val_dataset, test_dataset = {}, {}, {}
    
for dataset in dataset_list:
    train_dataset[dataset] = HateDataset(train_encodings[dataset], train_labels[dataset])
    val_dataset[dataset] = HateDataset(val_encodings[dataset], val_labels[dataset])
    test_dataset[dataset] = HateDataset(test_encodings[dataset], test_labels[dataset])

## Train Models

In [ ]:
# check CUDA availability

if torch.cuda.is_available():
    print('CUDA available')
    print(torch.cuda.get_device_name())
    print(torch.cuda.device_count(), 'GPUs')
else:
    print('CUDA unavailable')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Define training arguments
training_args = {}

for dataset in dataset_list:
    training_args[dataset] = TrainingArguments(
        save_steps = 10000,
        output_dir = './Models/BERT_{}_unweighted/Checkpoints'.format(dataset), # output directory
        num_train_epochs = 3,              # total number of training epochs
        per_device_train_batch_size = 16,  # batch size per device during training
        per_device_eval_batch_size = 64,   # batch size for evaluation
        evaluation_strategy = 'epoch',
        warmup_steps = 500,                # number of warmup steps for learning rate scheduler
        weight_decay = 0.01,               # strength of weight decay
        learning_rate = 5e-5,
        seed = 123
    )

In [ ]:
# define explicit model initialisation to allow for hyperparameter search
def model_init():
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
    # resize to match tokenizer length with special tokens added above
    model.resize_token_embeddings(len(tokenizer))
    return model

In [ ]:
# Instantiate unweighted trainer objects for each dataset
trainer = {}

for dataset in dataset_list:
    if dataset == 'combined':
        trainer[dataset] = Trainer(                       
            args=training_args[dataset],                  
            train_dataset=train_dataset[dataset],         
            eval_dataset=val_dataset[dataset],            
            model_init = model_init
        )

In [ ]:
# Fine-tune models for each dataset
for dataset in trainer:
    print('Training unweighted {} BERT model'.format(dataset))
    trainer[dataset].train()

## Save Model and Tokenizer

In [ ]:
for dataset in trainer:    
    trainer[dataset].save_model('./Models/BERT_{}_unweighted/Final'.format(dataset))
    tokenizer.save_pretrained('./Models/BERT_{}_unweighted/Final'.format(dataset))

## Reload Models
So that models can be evaluated on test set even after kernel resets

In [ ]:
# load fine-tuned models

def load_model(dataset):
    model =  BertForSequenceClassification.from_pretrained(f'./Models/BERT_{dataset}_unweighted_120221/Final')
    trainer = Trainer(
        model = model,
        args = TrainingArguments(
            output_dir=f'./Models/BERT_{dataset}_unweighted_120221/Test',
            per_device_eval_batch_size = 64)
    )
    return trainer

trainer = {}

for dataset in ['combined']:
    trainer[dataset] = load_model(dataset)

## Evaluate Models on Test Data

In [ ]:
# Evaluate each model on its corresponding test set

results = {}

for dataset in trainer:
    print('Evaluating unweighted {} BERT model on test data'.format(dataset))
    results[dataset] = trainer[dataset].predict(test_dataset[dataset])
    for metric in results[dataset].metrics:
        print(metric, results[dataset].metrics['{}'.format(metric)])
    print()

In [ ]:
# write predictions to series
pred_labels={}

for dataset in trainer:

    preds=[]
    
    for row in results[dataset][0]:
        preds.append(int(np.argmax(row)))
    
    pred_labels[dataset] = pd.Series(preds)

# print classification reports for each model

for dataset in trainer:
        print(dataset.upper())
        print(classification_report(test_labels[dataset],pred_labels[dataset]))
        print()

In [ ]:
# f1 scores
for dataset in trainer:
        print(dataset.upper())
        for average in ['micro', 'macro', 'weighted']:
            print('{} F1 score: {:.2%}'.format(average, f1_score(test_labels[dataset],pred_labels[dataset], average=average)))
        print()

In [ ]:
# distribution of predictions
for dataset in trainer:
        print(dataset.upper())
        print(pred_labels[dataset].value_counts())
        print()